<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 1 / NOTE 4*

---

# N414. Hyperparameters

## 하이퍼파라미터(Hyperparameter) 튜닝으로 성능 올리기


신경망에서는 신경써야 할 **<font color="ff6f61">하이퍼파라미터(Hyperparameter)</font>**가 굉장히 많습니다.<br/>
지금까지 다뤄온 머신러닝 알고리즘은 많아야 20개 정도의 하이퍼파라미터를 탐색하면 되었습니다.<br/>
하지만 신경망은 층을 깊게 쌓을수록 조정해주어야 할 하이퍼파라미터가 훨씬 더 많아지게 됩니다.

**하이퍼파라미터 조정(Tuning)**은 모델 성능에 엄청난 영향을 끼치는 요소이기 때문에 시간이 많이 소요되더라도 반드시 해주어야 합니다.<br/>
좋은 하이퍼파라미터를 찾기란 결코 쉽지 않습니다.<br/>
운좋게도 임의로 입력한 하이퍼파라미터가 만족스런 성능을 보일 수는 있지만 **'기도메타'가 언제나 우리에게 성공을 보장하지는 않죠.**<br/>
그렇다면 결정한 하이퍼파라미터로 구축한 모델이 좋은 성능을 보이는지를 어떻게 알 수 있을까요?

<img src="https://i.imgur.com/1MMrBS5.png" height = "200"/>


지난 강의에서 위 그림을 보며 '딥러닝도 머신러닝'임을 말씀드렸는데요.

머신러닝 알고리즘을 다룰 때에 일반적인 모델의 성능을 평가하기 위해서 **<font color="ff6f61">교차 검증(Cross-Validation)</font>**을 사용했던 것처럼<br/>
신경망도 교차 검증을 사용하여 일반화 성능을 평가할 수 있습니다.

아래 코드를 통해 신경망에 교차 검증을 적용하는 방법에 대해 알아보겠습니다.

### 신경망으로 Boston 집값 데이터 예제 해결하기

보스턴 집값 데이터셋(**`boston_housing`**) 예제를 신경망으로 풀어보겠습니다.<br/>
문제를 푸는 과정에서 교차 검증을 적용하여 풀어보도록 하겠습니다!

1. **데이터셋을 불러옵니다.**

In [ ]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


### 신경망에 교차 검증(Cross-Validation) 적용해보기

> ❗️ ***머신러닝에서 배운 교차 검증이 기억이 잘 안난다면 Section 2 로 돌아가 해당 내용을 복습해주세요!***


2. **필요한 라이브러리를 import 합니다.**

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import tensorflow as tf
import os

3. **`KFold`를 통해 학습 데이터셋을 몇 개(k)로 나눌지를 결정합니다.**

아래에서는 많이 사용되는 k인 5로 설정해보도록 하겠습니다.

In [ ]:
kf = KFold(n_splits = 5)
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) 

x_train.shape

(404, 13)

> ❓ ***`KFold`와 `StratifiedKFold`의 차이는 무엇일지 다시 떠올려봅시다.<br/>
어떤 경우에 `KFold`가 아닌 `StratifiedKFold`를 써주어야 할까요?***

In [ ]:
y_train[:5]

array([15.2, 42.3, 50. , 21.1, 17.7])

> ❗️ ***아래부터 등장하는 코드는 고의적으로 에러를 발생하도록 쓰여 있습니다. 설명을 충분히 읽으면서 실행해 주세요!***


In [ ]:
training_data = x_train.iloc[train_index]
validation_data = x_train.iloc[val_index]

# for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
#   training_data = x_train.iloc[train_index]
#   validation_data = x_train.iloc[val_index]

AttributeError: ignored

위처럼 **Numpy array 에서는 `.iloc` 을 쓸 수 없겠죠?**<br/>
그러니 **`pd.DataFrame()`** 을 이용해보겠습니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]), y_train):
    training_data = x_train.iloc[train_index]
    validation_data = x_train.iloc[val_index]
    training_y = y_train.iloc[train_index]
    validation_y = y_train.iloc[val_index]

아래 코드에서는 모델을 불러오는데 에러가 납니다! 무엇 때문에 나는 에러일까요?

In [ ]:
model = Sequential()

NameError: ignored

에러명을 살펴보면 `NameError: name 'Sequential' is not defined` 입니다.<br/>
**`Sequential`이 defined 되지 않았다는 뜻이므로 해당 패키지(`Sequential`)를 import** 해주어 해결해봅시다.

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

이번에는 Dense를 추가해줍니다.

In [ ]:
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

NameError: ignored

에러명 `NameError: name 'Dense' is not defined` 을 살펴보니 동일한 에러임을 알 수 있습니다.<br/>
**같은 유형의 에러이므로 같은 방법으로 해결**해보겠습니다.

In [ ]:
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))


model.compile(loss='mean_squared_logarithmic_error',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(training_data, training_y, epochs=2)

Epoch 1/2
11/11 [==============================] - 1s 3ms/step - loss: 0.6799 - accuracy: 0.0000e+00
Epoch 2/2
11/11 [==============================] - 0s 2ms/step - loss: 0.1771 - accuracy: 0.0000e+00


위 코드까지 모델이 잘 돌아가는 것을 확인하였습니다.

이제는 **교차 검증(Cross-Validation)을 적용할 차례**입니다.<br/>
다시 학습 데이터셋(**`x_train, y_train`**)을 k개 의 set으로 나누어주겠습니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0]),y_train):
    training_data = x_train.iloc[train_index, :]
    training_data_label = y_train.iloc[train_index]
    validation_data = x_train.iloc[val_index, :]
    validation_data_label = y_train.iloc[val_index]

다시 모델을 학습시켜줍니다.

In [ ]:
model.fit(training_data, training_data_label,
			epochs=10,
            batch_size=64,
			validation_data=(validation_data, validation_data_label),
          )

Epoch 1/10
6/6 [==============================] - 0s 46ms/step - loss: 0.1255 - accuracy: 0.0000e+00 - val_loss: 0.1453 - val_accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 0s 7ms/step - loss: 0.1206 - accuracy: 0.0000e+00 - val_loss: 0.0984 - val_accuracy: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 0s 9ms/step - loss: 0.1060 - accuracy: 0.0000e+00 - val_loss: 0.0939 - val_accuracy: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 0s 7ms/step - loss: 0.0924 - accuracy: 0.0000e+00 - val_loss: 0.0979 - val_accuracy: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 0s 7ms/step - loss: 0.0957 - accuracy: 0.0000e+00 - val_loss: 0.0948 - val_accuracy: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 0s 8ms/step - loss: 0.0881 - accuracy: 0.0000e+00 - val_loss: 0.0916 - val_accuracy: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 0s 8ms/step - loss: 0.0888 - accuracy: 0.0000e+00 - val_loss: 0.1016 - va

데이터가 잘 나누어져 들어갔는지 확인해봅니다. 

In [ ]:
print(training_data[:2])
print(training_data.shape)

        0     1     2    3      4   ...   8      9     10      11     12
0  1.23247   0.0  8.14  0.0  0.538  ...  4.0  307.0  21.0  396.90  18.72
1  0.02177  82.5  2.03  0.0  0.415  ...  2.0  348.0  14.7  395.38   3.11

[2 rows x 13 columns]
(324, 13)


In [ ]:
training_data_label[:2]

,0
0,15.2
1,42.3


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(training_data, training_data_label,
          epochs=10,
          batch_size=32,
          )

Epoch 1/10
11/11 [==============================] - 0s 2ms/step - loss: 89.6655
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 57.5176
Epoch 3/10
11/11 [==============================] - 0s 2ms/step - loss: 55.1179
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: 48.7623
Epoch 5/10
11/11 [==============================] - 0s 2ms/step - loss: 45.9350
Epoch 6/10
11/11 [==============================] - 0s 2ms/step - loss: 48.0356
Epoch 7/10
11/11 [==============================] - 0s 2ms/step - loss: 57.1652
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: 48.8794
Epoch 9/10
11/11 [==============================] - 0s 3ms/step - loss: 55.9871
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: 52.9977


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(training_data, training_data_label,
          epochs=10,
          batch_size=32,
          )

Epoch 1/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4154
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: -321.4153
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: -321.4153
Epoch 4/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 5/10
11/11 [==============================] - 0s 2ms/step - loss: -321.4153
Epoch 6/10
11/11 [==============================] - 0s 2ms/step - loss: -321.4154
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 8/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153
Epoch 9/10
11/11 [==============================] - 0s 4ms/step - loss: -321.4153
Epoch 10/10
11/11 [==============================] - 0s 3ms/step - loss: -321.4153


In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test mse:", results)

4/4 [==============================] - 0s 4ms/step - loss: -336.6793
test loss, test mse: -336.67926025390625


이제 한 번에 테스트를 수행해보겠습니다. 

In [ ]:
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

for train_index, val_index in kf.split(np.zeros(x_train.shape[0])):
    training_data = x_train.iloc[train_index, :]
    training_data_label = y_train.iloc[train_index]
    validation_data = x_train.iloc[val_index, :]
    validation_data_label = y_train.iloc[val_index]

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train,
              epochs=10,
              batch_size=32,
              validation_data = (validation_data, validation_data_label),
              )
    
    results = model.evaluate(x_test, y_test, batch_size=32)
    print("test loss, test mse:", results)

Epoch 1/10
13/13 [==============================] - 1s 15ms/step - loss: 87.9968 - val_loss: 61.2588
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 56.3914 - val_loss: 65.1370
Epoch 3/10
13/13 [==============================] - 0s 6ms/step - loss: 48.1239 - val_loss: 51.5774
Epoch 4/10
13/13 [==============================] - 0s 4ms/step - loss: 45.7609 - val_loss: 35.4846
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 40.3047 - val_loss: 35.9572
Epoch 6/10
13/13 [==============================] - 0s 6ms/step - loss: 40.5311 - val_loss: 35.7054
Epoch 7/10
13/13 [==============================] - 0s 6ms/step - loss: 42.6978 - val_loss: 35.1698
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 41.6586 - val_loss: 39.4655
Epoch 9/10
13/13 [==============================] - 0s 6ms/step - loss: 41.4178 - val_loss: 45.0892
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 47.9519
test loss, test mse:

교차 검증을 통해서 모델을 돌릴 수 있는 것까지 확인해보았습니다.

## 신경망에서의 하이퍼 파라미터 튜닝

교차 검증 방법을 익혔으니 이제 본격적으로 하이퍼파라미터 튜닝을 시도해보겠습니다.<br/>
머신러닝(Section 2)에서 공부하셨던 것처럼 흔히 사용되는 하이퍼파라미터 튜닝 방법에는 아래와 같은 것들이 있습니다.

### 하이퍼파라미터 튜닝 방식의 종류

1. **"Babysitting"(육아) 혹은 "Grad Student Descent"(대학원생 갈아넣기)**

    다윈의 진화론을 아시나요? 진화론에서는 '자연 선택'이란 단어가 진화를 주도했다고 말하곤 합니다.<br/>
하지만 하이퍼 파라미터 선택은 자연이 해주지 않습니다. 그렇다면 우리가 직접 하는 수 밖에 없겠죠?<br/>
이전 프로젝트나 이번 스프린트에서 모델 성능을 높이기 위해 여러 숫자를 직접 넣어보며 하이퍼 파라미터를 수없이 조정했다면,<br/>
첫 번째 방법을 수행했다고 말할 수 있겠습니다.

    100% **<font color="ff6f61">수작업(Manual)</font>**으로 파라미터를 수정하는 방법입니다.<br/>
학계에서 논문을 출간할 수 있을 정도로 놀라운 정확도를 보여주는 하이퍼파라미터의 수치를 찾아내기 위해 쓰는 방법이죠.<br/>
이를 위해서 실험자의 경험이나 도메인 지식이 필요하기도 합니다.<br/>
~~*(물론 지도교수님들이 이 걸 직접 하시진 않습니다, 교수님의 시간은 소중하니까요...)*~~

2. **Grid Search**

    하지만 언제까지나 이렇게 하나하나 수작업으로만 시도해 볼 수는 없겠죠.<br/>
1번 방식을 자동화한 방법이 바로 **<font color="ff6f61">"Grid Search"</font>**입니다.<br/>
이 방법에서는 하이퍼파라미터마다 탐색할 지점을 정해주면 모든 지점에 해당하는 조합을 알아서 수행합니다.

    Grid Search는 학습을 실행한 뒤 한참 놀다오면 되는 매우 편한 방법이지만 **장점만 있는 것은 아닙니다.**<br/>
범위를 너무 많이 설정하면 '좀 놀다 오면 끝나는' 수준을 넘어 '수료하고 취직을 하고 나서도 끝나지 않을 수도' 있는데요.<br/>
만약 5개의 파라미터에 대해 각각 5개의 지점을 지정해주면 Grid Search는 총 $5^5=3,125$ 번의 모델 학습을 진행하게 됩니다.<br/>
여기에 5번의 교차 검증까지 진행한다면 모델은 $3,125 \times 5 = 15,625$ 번이나 학습을 수행합니다.<br/>
모델 한 번 학습에 10분만 걸린다고 쳐도 **3달 반**이 걸리는 무시무시한 작업입니다. 실제로 이런 일은 없어야겠죠?

    그렇기 때문에 Grid Search 로 너무 많은 하이퍼파라미터 조합을 찾으려고 하지 않는 것이 좋습니다.<br/>
1개, 혹은 최대 2개 정도의 파라미터 최적값을 찾는 용도로 적합합니다.<br/>
굳이 많은 하이퍼파라미터 조합을 시도할 필요는 없습니다.<br/>
모델 성능에 **보다 직접적인 영향을 주는 하이퍼파라미터가 따로 있기 때문**인데요.<br/>
이러한 파라미터만 제대로 튜닝해서 최적값을 찾은 후 나머지 하이퍼파라미터도 조정해나가면 못해도 90% 이상의 성능을 확보할 수 있습니다.<br/>
이런 식으로 하나씩 접근하다 보면 적어도 무한루프가 발생하는 위험은 줄일 수 있습니다.

3. **Random Search**

    **<font color="ff6f61">"Random Search"</font>** 는 무한 루프라는 Grid Search의 단점을 해결하기 위해 나온 방법입니다.<br/>
Random Search 는 지정된 범위 내에서 무작위로 모델을 돌려본 후 최고 성능의 모델을 반환합니다.<br/> 시도 횟수를 정해줄 수 있기 때문에 Grid Search 에 비해서 훨씬 적은 횟수로도 끝마칠 수 있겠죠?

    Grid Search 에서는 파라미터의 중요도가 모두 동등하다고 가정합니다.<br/>
하지만 위에서 알아본 것처럼 실제로 더 중요한 하이퍼파라미터가 있는데요.<br/>
Random Search 는 **상대적으로 중요한 하이퍼파라미터에 대해서는 탐색을 더 하고, 덜 중요한 하이퍼파라미터에 대해서는 실험을 덜 하도록** 합니다.

    Random Search 는 절대적으로 완벽한 하이퍼파라미터를 찾아주지는 않는다는 단점을 가지고 있는데요.<br/>
하지만 Grid Search와 비교했을 때, 학습에 걸리는 시간이 훨씬 더 적다는 점으로도 Random Search의 의의를 찾을 수 있습니다.

> ❗️ ***아래 그림을 보면서 Grid Search 와 Random Search 의 차이에 대해서 생각해봅시다.***

<img src="https://i.imgur.com/qwySX8w.png" width="600">

4. **Bayesian Methods**

    "Baby sitting" 이나 "Grid Search" 등의 방식에서는 탐색 결과를 보고, 결과 정보를 다시 새로운 탐색에 반영하면 성능을 더 높일 수 있었습니다.<br/> **<font color="ff6f61">베이지안 방식(Bayesian Method)</font> 은 이렇게 이전 탐색 결과 정보를 새로운 탐색에 활용하는 방법**입니다.<br/>
그렇기 때문에 베이지안 방법을 사용하면 하이퍼파라미터 탐색 효율을 높일 수 있습니다.<br/>
`bayes_opt` 나 `hyperopt`와 같은 패키지를 사용하면 베이지안 방식을 적용할 수 있습니다.

### 튜닝 가능한 파라미터에는 어떤 것이 있을까요?

신경망에서 탐색해 볼 수 있는 하이퍼파라미터의 종류는 다음과 같습니다.

- 배치 크기(**`batch_size`**)
- 에포크(**`epochs`**)
- 옵티마이저(**`optimizers`**)
- 학습률(**`learning rate`**)
- 활성화 함수(**`activation`**)
- Regularization(**`weight decay, Dropout`** 등)
- 은닉층(Hidden layer)의 노드(Node) 수

> ❗️ ***실제로는 이보다 더 많은 하이퍼파라미터를 튜닝할 수 있습니다.<br/>
하지만 일단은 이정도만 기억해도 좋습니다. 반복하여 시도하다 보면 익숙해질 것입니다.***

### GridSearch 를 사용한 최적의 배치 사이즈 탐색하기

당뇨병 데이터셋을 신경망에 적용해보고 배치 사이즈를 여러 개로 조정하면서 최적의 배치 사이즈를 찾아보겠습니다.

1. **필요한 패키지를 import 합니다.**

In [26]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

2. **재현성을 위해 랜덤시드를 고정합니다**

In [27]:
numpy.random.seed(42)

3. **데이터셋을 불러온 후에 Feature 와 Label로 분리합니다.**

In [28]:
url ="https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

dataset = pd.read_csv(url, header=None).values

In [29]:
X = dataset[:,0:8]
Y = dataset[:,8]

4. **모델을 제작합니다.**

    추후 **`KerasClassifier`** 로 Wrapping 하기 위하여 신경망 모델을 함수 형태로 정의합니다. 

In [30]:
def create_model():
    model = Sequential()
    model.add(Dense(100, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

4. **`KerasClassifier` 로 wrapping 하여줍니다.**

In [31]:
model = KerasClassifier(build_fn=create_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


5. **하이퍼파라미터 탐색을 위한 탐색 범위를 설정한 후 `GridSearchCV` 를 지정하여 학습합니다.**

In [32]:
batch_size = [8, 16, 32, 64, 128]
param_grid = dict(batch_size=batch_size)

In [33]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

6. **최적의 결과를 낸 하이퍼파라미터와 각각의 결과를 출력해봅시다.**

In [34]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.6407011389732361 using {'batch_size': 16}
Means: 0.6315423130989075, Stdev: 0.014911532846494317 with: {'batch_size': 8}
Means: 0.6407011389732361, Stdev: 0.04977887055410017 with: {'batch_size': 16}
Means: 0.5677192091941834, Stdev: 0.058128465916732146 with: {'batch_size': 32}
Means: 0.6277056336402893, Stdev: 0.05704244478573418 with: {'batch_size': 64}
Means: 0.4517019033432007, Stdev: 0.10637838660895693 with: {'batch_size': 128}


## 라이브러리를 사용한 하이퍼파라미터 튜닝

### Keras Tuner 를 사용하여 하이퍼파라미터 탐색하기


**<font color="ff6f61">Keras Tuner</font>** 는 케라스 프레임워크에서 하이퍼파라미터를 튜닝하는 데 도움이 되는 라이브러리입니다.<br/>
Fashion MNIST 예제에 Keras Tuner를 적용하여 하이퍼파라미터 튜닝을 수행해보겠습니다.

1. **필요한 패키지를 import 합니다.**

In [35]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten

import tensorflow as tf
import IPython

2. **Keras Tuner를 설치한 후 import 합니다.**

Keras Tuner는 Colab에 내장된 패키지가 아니기 때문에 따로 설치를 해준 후에 import 하여줍니다.

In [36]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 98 kB 3.5 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


3. **데이터셋을 불러온 후에 정규화(Normalizing) 해줍니다.**

    Fashion MNIST 데이터셋을 불러온 후에 이미지를 0-1 사이의 값으로 정규화합니다.

In [38]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [39]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

4. **Model을 제작합니다.**

모델을 제작하고 탐색할 하이퍼파라미터 범위와 지점을 정의합니다.<br/>

이 과정에서 Model builder 함수(**`model_builder`**)를 지정하는 과정이 필요합니다.<br/>
`model_builder` 라는 함수를 정의하고 해당 함수 내부에서 모델 설계와 하이퍼파라미터 튜닝까지 모두 수행해보겠습니다.

- **탐색할 하이퍼파라미터와 범위**
    - 은닉층의 노드 수 : 32 부터 512 까지 32개씩 증가시키며 탐색
    - 학습률(Learning rate) : 0.01, 0.001, 0.0001 의 3개 지점을 탐색


In [40]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(Flatten(input_shape=(28, 28)))
  
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(Dense(units = hp_units, activation = 'relu'))
  model.add(Dense(10, activation='softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

5. **하이퍼파라미터 튜닝을 수행할 튜너(Tuner)를 지정합니다.**

Keras Tuner 에서는 **Random Search, Bayesian Optimization, Hyperband** 등의 최적화 방법을 수행할 수 있습니다.<br/>
아래에서는 **`Hyperband`** 를 통해서 튜닝을 수행해보도록 하겠습니다.

Hyperband 사용 시 Model builder function(**`model_builder`**), 훈련할 최대 epochs 수(**`max_epochs`**) 등을 지정해주어야 합니다.<br/>
Hyperband 는 리소스를 알아서 조절하고 조기 종료(Early-stopping) 기능을 사용하여 
높은 성능을 보이는 조합을 신속하게 통합한다는 장점을 가지고 있습니다.



In [41]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

6. **Callback 함수를 지정합니다.**

    하이퍼파라미터 탐색을 실행하기 전에 학습이 끝날 때마다 이전 출력이 지워지도록 콜백 함수를 정의해봅시다.

In [42]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

하이퍼파라미터 탐색을 수행합니다.

> ❗️ ***아래 코드를 통해 하이퍼파라미터 탐색을 수행하려면 약 20분의 시간이 필요합니다.<br/>
충분한 시간 여유를 가지고 수행해주세요.***


In [43]:
tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8533999919891357

Best val_accuracy So Far: 0.8804000020027161
Total elapsed time: 00h 19m 18s
INFO:tensorflow:Oracle triggered exit

하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 448 입니다.
최적의 학습 속도는 0.001 입니다.



7. **최고 성능을 보이는 하이퍼파라미터 조합으로 다시 학습을 진행해봅시다.**

In [44]:
model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 448)               351680    
                                                                 
 dense_3 (Dense)             (None, 10)                4490      
                                                                 
Total params: 356,170
Trainable params: 356,170
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.fit(X_train, y_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 10s 5ms/step - loss: 0.4778 - accuracy: 0.8297 - val_loss: 64.3352 - val_accuracy: 0.8294
Epoch 2/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3610 - accuracy: 0.8679 - val_loss: 52.7949 - val_accuracy: 0.8505
Epoch 3/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3236 - accuracy: 0.8809 - val_loss: 55.0612 - val_accuracy: 0.8547
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2990 - accuracy: 0.8886 - val_loss: 54.0619 - val_accuracy: 0.8621
Epoch 5/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2812 - accuracy: 0.8946 - val_loss: 50.8726 - val_accuracy: 0.8790
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2673 - accuracy: 0.9004 - val_loss: 63.9975 - val_accuracy: 0.8576
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2514 - accuracy: 0.9053 - val_loss: 62.4039 - val_accura